In [1]:
import findspark
findspark.init('/home/ubuntu/spark-2.4.5-bin-hadoop2.7')
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
import os
os.chdir('/home/ubuntu/data')

In [2]:
spark = SparkSession.builder.appName('als_recommendation').getOrCreate()

In [3]:
df = spark.read.csv('movielens_ratings.csv', inferSchema=True, header=True) 
df.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- userId: integer (nullable = true)



In [4]:
for row in df.head(5):
    print(row, '\n')

Row(movieId=2, rating=3.0, userId=0) 

Row(movieId=3, rating=1.0, userId=0) 

Row(movieId=5, rating=2.0, userId=0) 

Row(movieId=9, rating=4.0, userId=0) 

Row(movieId=11, rating=1.0, userId=0) 



In [5]:
df.groupBy('UserId').count().show()

+------+-----+
|UserId|count|
+------+-----+
|    28|   50|
|    26|   49|
|    27|   46|
|    12|   55|
|    22|   56|
|     1|   49|
|    13|   48|
|     6|   57|
|    16|   45|
|     3|   48|
|    20|   47|
|     5|   49|
|    19|   49|
|    15|   48|
|     9|   53|
|    17|   46|
|     4|   55|
|     8|   49|
|    23|   52|
|     7|   54|
+------+-----+
only showing top 20 rows



In [6]:
df.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [7]:
training, test = df.randomSplit([0.8, 0.2], seed=42)
training.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1202|              1202|              1202|
|   mean| 48.91514143094842|1.7920133111480865|14.624792013311149|
| stddev|29.214178581010305|1.1988071117072794| 8.590594802793422|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [8]:
test.describe().show()

+-------+-----------------+------------------+------------------+
|summary|          movieId|            rating|            userId|
+-------+-----------------+------------------+------------------+
|  count|              299|               299|               299|
|   mean|51.37792642140468|1.7023411371237458|13.414715719063546|
| stddev|27.75493754455142|1.1388625568976274| 8.538616435956683|
|    min|                0|               1.0|                 0|
|    max|               99|               5.0|                29|
+-------+-----------------+------------------+------------------+



In [9]:
als = ALS(maxIter=5, regParam=0.01, userCol='userId', itemCol='movieId', ratingCol='rating')
model = als.fit(training)
predictions = model.transform(test)

In [10]:
predictions.show()

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|     31|   1.0|    26| -2.025536|
|     31|   1.0|     5| 1.8553926|
|     31|   1.0|     4| 2.9258413|
|     31|   2.0|    25| 1.3368222|
|     31|   1.0|    18| 1.6312206|
|     85|   3.0|     1| 4.3166676|
|     85|   1.0|    13|0.83296514|
|     85|   3.0|     6| 3.4258354|
|     85|   1.0|    25|-3.6689477|
|     65|   1.0|    16| -2.930901|
|     65|   1.0|     2| 2.0725696|
|     78|   1.0|     1| 0.6628858|
|     78|   1.0|    19|0.48282775|
|     78|   1.0|    24| 1.5232875|
|     78|   1.0|     2|0.24984702|
|     34|   1.0|    28|-3.1651337|
|     34|   1.0|    16| -3.042254|
|     34|   1.0|    19| 0.2617333|
|     81|   1.0|     6| 1.0004904|
|     81|   2.0|     5|   1.89784|
+-------+------+------+----------+
only showing top 20 rows



In [11]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')
evaluator.evaluate(predictions)

2.0779146751868827

In [12]:
single_user = test.filter(test['userId']==11).select(['movieId', 'userId'])
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     10|    11|
|     20|    11|
|     22|    11|
|     23|    11|
|     27|    11|
|     32|    11|
|     39|    11|
|     59|    11|
|     61|    11|
|     66|    11|
|     67|    11|
|     75|    11|
|     81|    11|
|     86|    11|
|     88|    11|
|     94|    11|
|     97|    11|
+-------+------+



In [13]:
recommendation = model.transform(single_user)
recommendation.orderBy('prediction', ascending=False).show()

+-------+------+------------+
|movieId|userId|  prediction|
+-------+------+------------+
|     32|    11|    4.885915|
|     94|    11|   3.8838196|
|     20|    11|    3.561094|
|     88|    11|   3.4999099|
|     10|    11|   3.0745592|
|     22|    11|   2.9539418|
|     23|    11|   2.8915904|
|     27|    11|   2.7931235|
|     39|    11|    1.369153|
|     97|    11|    1.306675|
|     86|    11|   0.3990131|
|     66|    11|  0.22625133|
|     75|    11|  0.19973934|
|     59|    11|  0.14548719|
|     61|    11|-0.101518154|
|     67|    11|  -0.7973987|
|     81|    11|   -0.846668|
+-------+------+------------+

